## Connect to Open Library API and fetch book genres 
- for limited amount of books only (books with highest number of rankings)

In [10]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # For progress bar

Import custom function

In [11]:
import sys
sys.path.append('../functions/')

from fun_api import fetch_genre, fetch_all_genres

### Get list of ISBNs

In [12]:
# Load ISBNs from a CSV file or list
isbn_list = ["9780143126560", "9780062316097", "9781451648539"] 

### Import list of ISBN

In [13]:
df_ratings = pd.read_csv('../data/ISBN_count_rating.csv')
isbn_list = df_ratings['ISBN'].to_list()
isbn_list = list(set(isbn_list)) ## Keep only unique ISBNs

print(f"Number of ISBNs: {len(isbn_list)}")

df_ratings.head(5)

Number of ISBNs: 322158


,ISBN,count_rating
0,971880107,2264
1,316666343,1164
2,385504209,813
3,312195516,668
4,60928336,662


In [14]:
isbn_list_subset = isbn_list[:10] ##
print(f"Number of selected ISBNs: {len(isbn_list_subset)}")
isbn_list_subset[:5]

Number of selected ISBNs: 10


['037317196X', '395177111', '1931081514', '006091050X', '140316043']

#### Function to fetch book genre

In [15]:
fetch_genre("1586601989")

{'ISBN': '1586601989',
 'Title': 'Holy Bible',
 'Genres': "Manuscripts, Latin (Medieval and modern), Glossaries, vocabularies, Early works to 1800, Bible, Religion, Bibles, Biblical Studies - General, Religion / Bible / General, Bible - General, Commentaries, Authorized, Criticism, interpretation, Shorthand, Sacred books, Church of Jesus Christ of Latter-day Saints, BÍBLIA (INTERPRETAÇÃO), Juvenile literature, The Bible, Biblia, Christian life, Study and teaching, English Concordances, Bible, study and teaching, Jews, Antiquities, Apocryphal books (Old Testament), Education, Doctrines, Spiritual life, Religious life, New Testament, Peshitta, English Harmonies, Bible, commentaries, Introductions, Biography, Creation, Biblical teaching, History, Bible, criticism, interpretation, etc., Catholic Truth Society (Great Britain), Church year meditations, Lectionaries, Bible, theology, Bibel, Bijbel, Christianity, Spirituality, Bible. -- Old Testament, Bible. -- New Testament, Teenagers' use, E

In [16]:
# Run the function
results = fetch_all_genres(isbn_list_subset, max_workers=10)

Fetching Genres: 100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


In [17]:
results.head(5)

,ISBN,Title,Genres
0,033344714X,Chewing-gum Rescue and Other Stories (M Books),"Short stories, Juvenile fiction, Children's fi..."
1,006091050X,I never told anyone,"Child sexual abuse, Biography, Victims of crim..."
2,8401423201,Poemas - Miguel Hernandez,Unknown Genre
3,762721987,Not Found,Not Found
4,395177111,Not Found,Not Found


In [20]:
### Filter where Genre is not 'Not Found'
results = results[results['Genres'] != 'Not Found']

print(f"Number of ISBNs with Genre: {len(results)}")

Number of ISBNs with Genre: 5


#### Save results to CSV

In [18]:
results.to_csv("interm_outputs/book_genres_API.csv", index=False)

print("✅ Data saved to book_genres.csv")

✅ Data saved to book_genres.csv
